In [2]:
import pandas as pd
import numpy as np
import requests
from IPython.display import HTML, display
from bs4 import BeautifulSoup
from bs4 import Comment
import json
import html
import re
import urllib.parse

import dask.dataframe as dd
from dask.multiprocessing import get

# !pip install swifter --user
import swifter
from pandas.io.json import json_normalize

from joblib import Parallel, delayed
from multiprocessing import cpu_count

import threading

cores = cpu_count() #Number of CPU cores on your system
fracciones = cores * 2

C:\Users\matias\AppData\Roaming\Python\Python36\site-packages\tqdm\autonotebook\__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
#configuro proxy y headers
url_proxy = 'https://free-proxy-list.net/anonymous-proxy.html'
response_pxy = requests.get(url_proxy)
pxySoup = BeautifulSoup(response_pxy.text, 'html.parser')
rowsPxy = []
headPxy = []
for tablePart in pxySoup.find_all(attrs={'id':'proxylisttable'})[0].find_all(True, recursive=False):
    if(tablePart.name == 'thead'):
        for row in tablePart.find_all(True, recursive=False):
            headPxy = [tr.text for tr in row]
    if(tablePart.name == 'tbody'):
        for row in tablePart.find_all(True, recursive=False):
            rowsPxy.append([tr.text for tr in row])
proxiesDf = pd.DataFrame(data=rowsPxy, columns=headPxy)
headersChrome = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
}
def getTimeResponse(row):
    proxiesTemp = {
        "http": row['IP Address']
    }
    responseA = requests.post('https://www.twitter.com', headers=headersChrome, proxies=proxiesTemp)
    timeA = responseA.elapsed.total_seconds()
    print(timeA)
    return timeA
proxiesDf.head()

,IP Address,Port,Code,Country,Anonymity,Google,Https,Last Checked
0,190.147.137.66,46290,CO,Colombia,elite proxy,no,yes,3 seconds ago
1,178.88.119.150,37139,KZ,Kazakhstan,elite proxy,no,yes,3 seconds ago
2,134.236.255.6,53995,TH,Thailand,elite proxy,no,no,3 seconds ago
3,103.9.190.206,34272,KH,Cambodia,elite proxy,no,yes,3 seconds ago
4,105.27.116.46,41109,ZA,South Africa,elite proxy,no,yes,3 seconds ago


In [4]:
def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    return out

In [5]:
proxiesWithTime = proxiesDf.copy()
proxiesWithTime.loc[:, 'timeResponse'] = np.NaN

def parallCheckResponses(arg):
    cuenta_errores = 0
    proxiesWithTime.loc[arg.index.values, 'timeResponse'] = arg.apply(getTimeResponse, axis=1)
    return False

Parallel(
    n_jobs=fracciones, backend="threading",
)(map(
    delayed(parallCheckResponses),
    chunkIt(proxiesWithTime, fracciones)
))
proxiesWithTime

0.669774
0.670296
0.687373
0.690519
0.686534
0.745094
0.75829
0.712888
0.816879
0.7150080.750524

0.779317
0.786388
0.8698940.846378
0.8633830.854717


0.8515380.8645750.85458

0.870642
0.885977
1.067616

0.688212
0.70892
0.670342
0.664459
0.685711
0.668659
0.731322
0.717011
0.722941
0.726682
0.81311
0.744261
0.745124
0.748316
0.745327
0.765355
0.756378
0.827328
0.84552
0.789073
0.809644
0.861846
0.861587
0.8717
0.923675
0.656528
0.6750430.737094

0.620414
0.726779
0.738112
0.704768
0.691584
0.723604
0.755369
0.767056
0.724368
0.726025
0.801157
0.733583
0.826516
0.76262
0.851798
0.8401471.070614

0.850943
0.9035
0.831539
1.680999
0.745023
0.784715
0.784874
0.720747
0.698384
0.665372
0.669587
0.674501
0.669352
0.70026
0.705338
0.762523
0.719737
0.80439
0.80205
0.743416
0.809284
0.755634
0.77257
0.773885
0.8359430.849454

0.84362
0.912069
0.863380.907388

0.712242
0.743191


,IP Address,Port,Code,Country,Anonymity,Google,Https,Last Checked,timeResponse
0,190.147.137.66,46290,CO,Colombia,elite proxy,no,yes,3 seconds ago,0.745094
1,178.88.119.150,37139,KZ,Kazakhstan,elite proxy,no,yes,3 seconds ago,0.731322
2,134.236.255.6,53995,TH,Thailand,elite proxy,no,no,3 seconds ago,0.738112
3,103.9.190.206,34272,KH,Cambodia,elite proxy,no,yes,3 seconds ago,0.669587
4,105.27.116.46,41109,ZA,South Africa,elite proxy,no,yes,3 seconds ago,0.712888
5,103.194.192.25,46397,IN,India,elite proxy,no,yes,3 seconds ago,0.726682
6,94.230.148.201,50823,CZ,Czech Republic,elite proxy,no,yes,3 seconds ago,0.767056
7,77.85.169.71,52770,BG,Bulgaria,elite proxy,no,no,3 seconds ago,0.669352
8,94.230.159.145,46749,CZ,Czech Republic,elite proxy,no,yes,3 seconds ago,0.688212
9,86.100.72.118,44852,LT,Lithuania,elite proxy,no,yes,3 seconds ago,0.923675


In [6]:
bestProxies = proxiesWithTime.sort_values(by='timeResponse')
bestProxies = bestProxies[:50]
def getProxie():
    return {
        "http": bestProxies.sample(1)['IP Address'].values[0]
    }
bestProxies.head()

,IP Address,Port,Code,Country,Anonymity,Google,Https,Last Checked,timeResponse
72,94.230.148.233,61576,CZ,Czech Republic,elite proxy,no,yes,3 seconds ago,0.620414
93,110.168.24.76,8118,TH,Thailand,anonymous,no,no,3 seconds ago,0.656528
38,190.124.34.221,61670,NI,Nicaragua,elite proxy,no,yes,3 seconds ago,0.664459
28,203.210.84.122,61218,ID,Indonesia,elite proxy,no,yes,3 seconds ago,0.665372
71,181.112.40.114,55829,EC,Ecuador,elite proxy,no,yes,3 seconds ago,0.668659


In [24]:
dataTweets = pd.read_pickle('../datasets/TweetsDataset.pkl')
scrapedUsers = pd.read_pickle('../datasets/TweetsScrapedUsers.pkl')
toScrap = pd.read_pickle('../datasets/UsersDataset.pkl')

print(len(toScrap), 'usuarios totales')
print(len(scrapedUsers), 'usuarios escrapeados')

toScrap.index = toScrap.index.map(int)
toScrap = toScrap.loc[toScrap.index.difference(scrapedUsers.index.map(int)).values]

print(len(toScrap), 'usuarios restantes')
print(len(toScrap) * 15 * 20, 'tweets restantes')
print(len(toScrap) * 15, 'paginas restantes')


15482 usuarios totales
1489 usuarios escrapeados
13993 usuarios restantes
4197900 tweets restantes
209895 paginas restantes


In [25]:
toScrap.sample(5)

,id,screen_name,complete_name,tweet_count,description,location,birthdate,url,listed_count,created_at,crawled_at,favs_count,followers_count,following_count,verified
index,,,,,,,,,,,,,,,
132768793,132768793,Lauletras,🤔Laura Grados,58798,lauletras@utero.pe,Lima-Perú,False,utero.pe,0,2010-04-01 00:00:00,2019-07-02 15:10:04.863988,7984,22741,1219,False
1027667946741608450,1027667946741608450,SebasCuervo3,SebasCuervo,4962,Es más fácil engañar a la gente que convencer...,"Buenos Aires, Argentina",False,False,0,2018-08-01 00:00:00,2019-07-02 19:30:13.855004,24477,391,264,False
188857646,188857646,OneJensen,Onejensen 🇪🇺,33587,"Tatuando cuando me dejan, tuiteando cuándo pue...",De aquí,False,instagram.com/onejensen,6,2010-09-01 00:00:00,2019-07-02 19:07:04.475878,30965,29300,589,False
74205573,74205573,miqueliceta,Miquel Iceta Llorens / ❤️,54330,Primer secretari del PSC i president del grup ...,Barcelona,1960,False,4,2009-09-01 00:00:00,2019-07-02 20:24:58.369572,1222,98438,4105,True
27280886,27280886,NoticiasTribuna,Tribuna Noticias,229906,"Medio informativo plural, objetivo y veraz. Pr...","Puebla, México",False,tribunanoticias.mx,1,2009-03-01 00:00:00,2019-06-30 21:41:49.771536,1135,66026,356,False


In [26]:
# dataTweets = pd.DataFrame(columns=[
#     'tweet_id',
#     'user_id',
#     'screen_name',
#     'complete_name',
#     'text',
#     'hashtags',
#     'mencioned_user_id',
#     'mencioned_screen_name',
#     'is_reply',
#     'reply_screen_name',
#     'reply_user_id',
#     'all_reply_screen_name',
#     'all_reply_user_id',
#     'is_retweet',
#     'retweet_user_id',
#     'retweet_screen_name',
#     'retweet_complete_name',
#     'retweet_count',
#     'reply_count',
#     'favorite_count',
#     'num_hashtags',
#     'num_urls',
#     'num_mentions',
#     'created_at',
#     'crawled_at'
# ])

lock = threading.RLock()

def getTweetData(tweet):
    print('process')
    text = tweet.findAll(attrs={'class':'content'})[0].findAll(attrs={'class':'tweet-text'})[0].text
    hashtagsSrc = tweet.findAll(attrs={'class':'content'})[0].findAll(attrs={'class':'tweet-text'})[0].\
        findAll(attrs={'class':'twitter-hashtag'})
    hashtags = []
    if(len(hashtagsSrc)>0):
        hashtags = [x.text[1:] for x in hashtagsSrc]
    amountHashtags = len(hashtags)
    retweetData = tweet.findAll(attrs={'class':'js-retweet-text'})
    isRetweeted = False
    retweet_userid = np.NaN
    retweet_displayName = np.NaN
    retweet_username = np.NaN
    
    username = False
    userid = False
    displayName = False
    if(len(retweetData) > 0):
        isRetweeted = True
        retweetUser = retweetData[0].findAll(attrs={'class':'js-user-profile-link'})[0]
        
        userid = retweetUser.attrs['data-user-id']
        displayName = retweetUser.findAll('b')[0].text
        username = retweetUser.attrs['href'][1:]
    else:
        username = tweet.findAll(attrs={'class':'username'})[0].text[1:]
        userid = tweet.findAll(attrs={'class': 'js-action-profile'})[0].attrs['data-user-id']
        displayName = tweet.findAll(attrs={'class':'js-stream-tweet'})[0].attrs['data-name']
    repply_data = tweet.findAll(attrs={'class':'ReplyingToContextBelowAuthor'})
    is_reply = False
    all_reply_screen_name = []
    all_reply_user_id = []
    if(len(repply_data)==1):
        repply_data = repply_data[0]
        is_reply = True
        for repplied_user in repply_data.findAll('a'):
            all_reply_screen_name.append(repplied_user.attrs['href'][1:])
            all_reply_user_id.append(repplied_user.attrs['data-user-id'])
    reply_count =  tweet.findAll(attrs={'class':'ProfileTweet-action--reply'})[0]\
    .findAll(attrs={'class':'ProfileTweet-actionCount'})[0].attrs['data-tweet-stat-count']
    retweet_count = tweet.findAll(attrs={'class':'ProfileTweet-action--retweet'})[0]\
    .findAll(attrs={'class':'ProfileTweet-actionCount'})[0].attrs['data-tweet-stat-count']
    favorite_count = tweet.findAll(attrs={'class':'ProfileTweet-action--favorite'})[0]\
    .findAll(attrs={'class':'ProfileTweet-actionCount'})[0].attrs['data-tweet-stat-count']
    tweetContainer = tweet.findAll(attrs={'class':'js-tweet-text-container'})[0]
    mencioned_user_id = []
    mencioned_screen_name = []
    for mencioned in tweetContainer.findAll('a', {'data-mentioned-user-id': re.compile(r'.*')},):
        mencioned_user_id.append(mencioned.attrs['data-mentioned-user-id'])
        mencioned_screen_name.append(mencioned.attrs['href'][1:])
    links_count = len(tweetContainer.findAll(attrs={'class':'twitter-timeline-link'}))
    global dataTweets
    dataTweets.loc[int(tweet.attrs['data-item-id'])] = {
        'tweet_id': int(tweet.attrs['data-item-id']),
        'user_id': userid,
        'screen_name': username,
        'complete_name': displayName,
        'text': text,
        'hashtags': hashtags,

        'mencioned_user_id': mencioned_user_id,
        'mencioned_screen_name':mencioned_screen_name,

        'is_reply': is_reply,
        'reply_screen_name': np.NaN if len(all_reply_screen_name)==0 else all_reply_screen_name[0],
        'reply_user_id': np.NaN if len(all_reply_user_id)==0 else all_reply_user_id[0],
        'all_reply_screen_name': all_reply_screen_name,
        'all_reply_user_id': all_reply_user_id,

        'is_retweet': isRetweeted,
        'retweet_user_id': retweet_userid,
        'retweet_screen_name': retweet_username,
        'retweet_complete_name': retweet_displayName,

        'retweet_count': retweet_count,
        'reply_count': reply_count,
        'favorite_count': favorite_count,
        'num_hashtags': amountHashtags,
        'num_urls': links_count,
        'num_mentions': len(mencioned_screen_name),

        'created_at': pd.Timestamp(int(tweet.findAll(attrs={'class':'js-short-timestamp'})[0].attrs['data-time']), unit='s'),
        'crawled_at': pd.Timestamp.now()
    }

    
headersMore = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Host': 'twitter.com',
    'Accept-Language': 'es-AR,es;q=0.8,en-US;q=0.5,en;q=0.3',
    'Accept-Encoding': 'gzip, deflate, br',
    'X-Twitter-Active-User': 'yes',
    'X-Requested-With': 'XMLHttpRequest',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
}

def getTweetsOfUser(userId, userName):
    tweetUserCount = 0
    proxieUser = getProxie()
    pageResponse = requests.get('https://twitter.com/'+userName, headers=headersChrome, proxies=proxieUser)
    twData = BeautifulSoup(pageResponse.text, 'html.parser')
    initialStream = twData.findAll(attrs={'id':'timeline'})[0].findAll(attrs={'class':'stream-container'})[0]
    dataMax = initialStream.attrs['data-max-position']
    dataMin = initialStream.attrs['data-min-position']
    streamItems = initialStream.findAll(attrs={'id':'stream-items-id'})[0]
    tweetList = streamItems.findAll('li', recursive=False)
    tweetUserCount += len(tweetList)
    if(len(tweetList) == 0):
        print('NO TWEETS', userName)
        return False
    for tweet in tweetList:
        getTweetData(tweet)
    urlMore = 'https://twitter.com/i/profiles/show/'+userName+'/timeline/tweets'
    maxPages = 9
    lastMaxKey = dataMax
    for pageNum in range(maxPages):
#         if(pageNum % 2 == 0):
        print('PAG '+str(pageNum)+' de 9.', userName)
        moreResponseRaw = requests.get(
            urlMore,
            headers=headersMore,
            proxies=proxieUser,
            params= {
                'max_position': lastMaxKey,
                'reset_error_state': False,
                'include_available_features': 1,
            }
        )
        moreResponse = json.loads(moreResponseRaw.text)
        print('StartPage')
        moreTweets = BeautifulSoup(moreResponse['items_html'], 'html.parser')
        tweetList = moreTweets.findAll('li', recursive=False)
        tweetUserCount += len(tweetList)
        if(len(tweetList) == 0):
            break
        with lock:
            for tweet in tweetList:
                getTweetData(tweet)
        lastMaxKey = moreResponse['min_position']
        print('endPage')
    with lock:
        global scrapedUsers
        scrapedUsers.loc[userId] = [userId, userName, tweetUserCount]
        

getTweetsOfUser(162861099, 'magai7')

process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
PAG 0 de 9. magai7
StartPage
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
endPage
PAG 1 de 9. magai7
StartPage
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
endPage
PAG 2 de 9. magai7
StartPage
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
endPage
PAG 3 de 9. magai7
StartPage
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
process
endPage
PAG 4 de 9. magai7
StartPage
process
process

In [ ]:
tweetsErrors = 0
tweetsErrUsers = []

def saveResults():
    with lock:
        print('--CHECKPOINT--')
        global scrapedUsers
        global dataTweets
        scrapedUsers.to_pickle('../datasets/TweetsScrapedUsers.pkl')
        dataTweets.to_pickle('../datasets/TweetsDataset.pkl')

def parallTweets(args):
    usersTotal = len(args)
    currUser = 1
    for userId, userName in zip(args.id, args.screen_name):
        print('--------------'+str(currUser)+' de '+str(usersTotal)+'--------------')
        if(currUser % 2 == 0):
            # Descargar a un archivo cada 2 usuarios
            saveResults()
        currUser += 1
        try:
            getTweetsOfUser(userId, userName)
        except KeyboardInterrupt:
            raise
        except:
            global tweetsErrors
            global tweetsErrUsers
            print('ERR', userName)
            tweetsErrors += 1
            tweetsErrUsers.append(userName)
    saveResults()

try:
    Parallel(
        n_jobs=fracciones, backend="threading",
    )(map(
        delayed(parallTweets),
        chunkIt(toScrap[['id', 'screen_name']], 1)
    ))
except KeyboardInterrupt:
    raise

print('DONE')
print('ERRORS:',tweetsErrors)
print(tweetsErrUsers)

--------------1 de 13993--------------
NO TWEETS arielblanco
--------------2 de 13993--------------
--CHECKPOINT--
PAG 0 de 9. clarincom
PAG 1 de 9. clarincom
PAG 2 de 9. clarincom
PAG 3 de 9. clarincom
PAG 4 de 9. clarincom
PAG 5 de 9. clarincom
PAG 6 de 9. clarincom
PAG 7 de 9. clarincom
PAG 8 de 9. clarincom
--------------3 de 13993--------------
PAG 0 de 9. naroh
PAG 1 de 9. naroh
PAG 2 de 9. naroh
PAG 3 de 9. naroh
PAG 4 de 9. naroh


In [ ]:
print(len(scrapedUsers))
display(scrapedUsers.head())
print(len(dataTweets))
display(dataTweets.head())
saveResults()